## Square Attack

The Square Attack is a black-box method used to generate adversarial samples. Unlike 'white-box' approachs such as PGD or FGSM, the Square Attack does not require knowing model weights or gradients. 

While other black-box attack take many queries to perform attacks the square attack makes relatively few.  The attack works by taking repeated alterations in the shape of a square on the image, keeping it if it increases the loss of the model. The Square Attack, upon release, was successful enough that it even outperformed some existing white-box approaches on benchmarks.s

## 1. Loading and Processing Models and Images

In [ ]:
# IF YOU ARE IN COLAB OR HAVE NOT INSTALLED `xlab-security`
!pip install xlab-security # should not take more than a minute or two to run

In [ ]:
import xlab
from xlab.utils import prediction, process_image, show_image, CIFAR10
import torch
import numpy as np
import random
from huggingface_hub import hf_hub_download
from xlab.models import MiniWideResNet, BasicBlock
import torch

classes = CIFAR10().classes
IMG_PATH = 'car.jpg'

model = MiniWideResNet()
model_path = hf_hub_download(
    repo_id="uchicago-xlab-ai-security/tiny-wideresnet-cifar10",
    filename="adversarial_basics_cnn.pth"
)

model = torch.load(model_path, map_location='cpu', weights_only = False)

## 2. The $ L_\infty $ Attack

### Task 1
First, we will start by making a helper function for the $ L_\infty $ attack. This will be the function for creating the distribution used to alter squares. In this algorithm, two corner indices $ r, s $ are randomly sampled, and are used with the window size $ h $ to create a window, which is then altered. The perturbation $ p $, used to alter the pixels in the window, is randomly sampled between $ -2\epsilon $ and $ 2\epsilon $. Each colour channel is altered separately, with the same window being changed in each.

$$
\begin{array}{l}
\text{\bf Algorithm 2: Sampling distribution } P \text{ for } l_\infty\text{-norm} \\
\hline
\text{\bf Input: } \text{maximal norm } \epsilon, \text{ window size } h, \text{ image size } w, \text{ color channels } c \\
\text{\bf Output: } \text{New update } \delta \\[0.5em]
\delta \leftarrow \text{array of zeros of size } w \times w \times c \\
\text{sample uniformly } r, s \in \{0, \dots, w - h\} \subset \mathbb{N} \\
\text{\bf for } i = 1, \dots, c \text{ \bf do } \\
\quad \rho \leftarrow \text{Uniform}(\{-2\epsilon, 2\epsilon\}) \\
\quad \delta_{r+1:r+h, \: s+1:s+h, \: i} \leftarrow \rho \cdot \mathbf{1}_{h \times h} \\
\text{\bf end } \\
\hline
\end{array}
$$

<details>
<summary>💡 <b>Hint for Task #1</b></summary>

In our solution we use ```torch.zeros``` to create a null matrix of the right shape and then add the individual values.

</details>
<details>
<summary>💡 <b>Hint for Task #1</b></summary>

In our solution, we index for the window using:
```arr[a:b+c][d:e+f]```

</details>

<details>
<summary>🔐 <b>Solution for Task #1</b></summary>

```python
def l_inf_dist(epsilon, h, w, c):
    """Generates a sparse adversarial perturbation by adding uniform noise to a random square patch
    
    Args:
        epsilon (float): Maximum perturbation magnitude.
        h (int): Height and width of the square patch to perturb.
        w (int): Height and width of the full image.
        c (int): Number of color channels.
        
    Returns:
        [c, w, w]: Sparse perturbation tensor with random noise in a single square patch.
    """
    randy = torch.randint(0, w- h, (1,)) 
    randx = torch.randint(0, w- h, (1,)) 

    ps = (torch.rand(c,1,1) - 0.5) * 4 * epsilon
    delta = torch.zeros(c, w, w)
    delta[:,randy:randy+h, randx:randx+h] = ps

    return delta
```

</details>

In [ ]:
def l_inf_dist(epsilon, h, w, c):
    """Generates a sparse adversarial perturbation by adding uniform noise to a random square patch
    
    Args:
        epsilon (float): Maximum perturbation magnitude.
        h (int): Height and width of the square patch to perturb.
        w (int): Height and width of the full image.
        c (int): Number of color channels.
        
    Returns:
        [c, w, w]: Sparse perturbation tensor with random noise in a single square patch.
    """
    randy = torch.randint(0, w- h, (1,)) 
    randx = torch.randint(0, w- h, (1,)) 

    ps = (torch.rand(c,1,1) - 0.5) * 4 * epsilon
    delta = torch.zeros(c, w, w)
    delta[:,randy:randy+h, randx:randx+h] = ps

    return delta

In [ ]:
delta = l_inf_dist(0.1, 5, 50, 3)
_ = xlab.utils.plot_tensors([
        delta[0],
        delta[1],
        delta[2]
    ],
    ncols=3,
    titles=["red channel", "green channel", "blue channel"] 
)

In [ ]:
_ = xlab.tests.square.task1(l_inf_dist)

## 2.1 The Square Attack loop.
### Task 2
Now, we will use the distribution within the main loop of the  $ L_\infty $ attack. Similar code will be used to run the $ L_2 $ attack. Intuitively, this algorithm involves choosing a window size $ h $ using a schedule of sorts, and then altering the image using the above helper function, until the image is adversarial or a particular number of iterations is reached.

$$
\begin{array}{l}
\text{\bf Algorithm: Projected Gradient-Free Adversarial Attack} \\
\hline
\text{\bf Input: } \text{classifier } f, \text{ point } x \in \mathbb{R}^d, \text{ image size } w, \text{ number of color channels } c, \\
\quad l_p\text{-radius } \epsilon, \text{ label } y \in \{1, \dots, K\}, \text{ number of iterations } N \\
\text{\bf Output: } \text{approximate minimizer } \hat{x} \in \mathbb{R}^d \text{ of the problem stated in Eq. (1)} \\[0.5em]
\hat{x} \leftarrow \text{init}(x) \\ 
l^* \leftarrow L(f(x), y) \\
i \leftarrow 1 \\
\text{\bf while } i < N \text{ and } \hat{x} \text{ is not adversarial \bf do} \\
\quad h^{(i)} \leftarrow \text{side length of the square to modify (according to some schedule)} \\
\quad \delta \sim P(\epsilon, h^{(i)}, w, c, \hat{x}, x) \\
\quad \hat{x}_{\text{new}} \leftarrow \text{Project } \hat{x} + \delta \text{ onto } \{z \in \mathbb{R}^d : \|z - x\|_p \le \epsilon\} \cap [0, 1]^d \\
\quad l_{\text{new}} \leftarrow L(f(\hat{x}_{\text{new}}), y) \\
\quad \text{\bf if } l_{\text{new}} < l^* \text{ \bf then } \hat{x} \leftarrow \hat{x}_{\text{new}}, l^* \leftarrow l_{\text{new}} \\
\quad i \leftarrow i + 1 \\
\text{\bf end } \\
\hline
\end{array}
$$

<details>
<summary>💡 <b>Hint for Task #2</b></summary>

In our solution, we use the div operator, ```//``` for the floor function.
</details>


<details>
<summary>🔐 <b>Solution for Task #2</b></summary>

```python
def l_inf_square_attack(model, loss_fn, x, y, N, w=32, c=3, epsilon=15/1000, max_h = 6):
    """Generates adversarial example using iterative square patch perturbations with L-infinity constraints
    
    Args:
        model: Neural network model with forward() method.
        loss_fn: Loss function for optimization.
        x [c, w, w]: Input image tensor.
        y (int): True class label.
        N (int): Maximum number of optimization iterations.
        w (int): Image width and height.
        c (int): Number of color channels.
        epsilon (float): Maximum perturbation magnitude per pixel.
        max_h (int): Initial square patch size, gradually reduced during optimization.
    
    Returns:
        [c, w, w]: Adversarial image tensor with successful misclassification or best attempt.
    """
    x_hat = x
    loss = loss_fn(model(x), y)
    i = 1
    h = max_h
    while i < N and prediction(model, x_hat)[0] == y:
        if i % (N // max_h) == 0:
            if h > 2:
                h -= 1
        delta = l_inf_dist(h=h, epsilon=epsilon, w=w, c=c)
        x_new = x_hat + delta
        x_new = torch.clamp(x_new, 0, 1)
        loss_new = loss_fn(model(x_new), y)
        if loss_new > loss:
            loss = loss_new
            x_hat = x_new 
        i += 1
    return x_hat
```

</details>

A basic schedule could involve reducing h by one (from max_h) every N//h iterations until the value of 2 is reached. This reduces the size of the window over time, reducing the scale of the perturbations, simulating a sort of convergence onto an adversarial image.

In [ ]:
def l_inf_square_attack(model, loss_fn, x, y, N, w=32, c=3, epsilon=15/1000, max_h = 6):
    """Generates adversarial example using iterative square patch perturbations with L-infinity constraints
    
    Args:
        model: Neural network model with forward() method.
        loss_fn: Loss function for optimization.
        x [c, w, w]: Input image tensor.
        y (int): True class label.
        N (int): Maximum number of optimization iterations.
        w (int): Image width and height.
        c (int): Number of color channels.
        epsilon (float): Maximum perturbation magnitude per pixel.
        max_h (int): Initial square patch size, gradually reduced during optimization.
    
    Returns:
        [c, w, w]: Adversarial image tensor with successful misclassification or best attempt.
    """

    raise NotImplementedError("l_inf_square_attack hasn't been implemented.")

In [ ]:
_ = xlab.tests.square.task2(model, l_inf_square_attack)

In [ ]:
img = process_image(IMG_PATH)
label = prediction(model, process_image(IMG_PATH))[0]
loss_fn = torch.nn.CrossEntropyLoss()
x_adv = l_inf_square_attack(model, loss_fn, img, label, 500)
pred = prediction(model, x_adv)
print(f"{classes[pred[0]]} with probability {pred[1][0]:.4f}")
show_image(x_adv)

## 3. The $ L_2 $ Attack
While the main loop of this attack is the same as the above, the distribution is entirely different. We will begin by producing the helper functions for this, and then the function itself. As the name suggests, the $L_2$ attack minimizes the $L_2$ difference between the original image and the perturbed one, so it is inherently slower and more complex. 

The $L_2$ distribution creates two adjacent rectangles of pixels (forming a square). The pixels in each rectangle have opposite signs. Placed together, this has a successfully adversarial effect. We will first create some helper functions for this, before moving on to implementing the distribution itself. This is significantly harder to implement than the $L_\infty$ attack, and has been included as more of an implementation challenge than anything. This section can be skipped through if found too challenging, but it is a good way to test your skills and practice using torch functions.

### Task 3
Code the $M$ helper function for $L_2$ distribution. This function effectively creates a 'pyramidal' shape of values, with the highest value in the centre, and decreasing values radially around it. This is used to create adversarial perturbations—experiments show it to be an effective method.
$$
M(r, s) = \left\lfloor \frac{h_1}{2} \right\rfloor - 
\max\left\{
    \left| r - \left\lfloor \frac{h_1}{2} \right\rfloor - 1 \right|, 
    \left| s - \left\lfloor \frac{h_2}{2} \right\rfloor - 1 \right|
\right\}
$$


<details>
<summary>💡 <b>Hint for Task #3</b></summary>

In our solution, we use the div operator, ```//``` for the floor function.
</details>


<details>
<summary>🔐 <b>Solution for Task #3</b></summary>

```python
def M(r, s, h1, h2):
    """Computes the L-infinity distance from coordinates to the center of a rectangle
    
    Args:
        r (int): X coordinate.
        s (int): Y coordinate.
        h1 (int): Rectangle width.
        h2 (int): Rectangle height.
    
    Returns:
        int: Maximum absolute distance along either axis from point to rectangle center.
    """
    n = h1 // 2
    return max(abs(r - h1 // 2 - 1), abs(s - h2 // 2 - 1))
```

</details>

In [ ]:
def M(r, s, h1, h2):
    """Computes the L-infinity distance from coordinates to the center of a rectangle
    
    Args:
        r (int): X coordinate.
        s (int): Y coordinate.
        h1 (int): Rectangle width.
        h2 (int): Rectangle height.
    
    Returns:
        int: Maximum absolute distance along either axis from point to rectangle center.
    """
    
    raise NotImplementedError("M hasn't been implemented.")

In [ ]:
_ = xlab.tests.square.task3(M)

### Task 4
Code the eta helper function for the $L_2$ distribution. This, using the $M$ helper function, creates a rectangle which has smoothed, radially decreasing pixel values. 
$$
\eta_{r,s}^{(h_1,h_2)} = \sum_{k=0}^{M(r,s)} \frac{1}{(n+1-k)^2}
$$

<details>
<summary>💡 <b>Hint for Task #4</b></summary>

In our solution, we use a cache for faster computation, but this is not necessary.
</details>


<details>
<summary>🔐 <b>Solution for Task #4</b></summary>

```python
def eta(h1, h2):
    """Generates a weight matrix with harmonic series values based on distance from rectangle center
    
    Args:
        h1 (int): Rectangle width.
        h2 (int): Rectangle height.
    
    Returns:
        [h1, h2]: Weight matrix where values increase with distance from center using harmonic progression.
    """
    n = h1 // 2
    m_matrix = torch.tensor([[M(i, j, h1, h2) for i in range(h2)] for j in range(h1)])
    max_value = torch.max(m_matrix)
    cache = []
    t_sum = 0
    for i in range(max_value):
        if n+1-i != 0: 
            t_sum += 1 / (n+1-i)
        cache.append(t_sum)
    cache = torch.tensor(cache)
    eta_matrix = cache[m_matrix - 1]
    return eta_matrix
```

</details>

In [ ]:
def eta(h1, h2):
    """Generates a weight matrix with harmonic series values based on distance from rectangle center
    
    Args:
        h1 (int): Rectangle width.
        h2 (int): Rectangle height.
    
    Returns:
        [h1, h2]: Weight matrix where values increase with distance from center using harmonic progression.
    """

    raise NotImplementedError("eta hasn't been implemented.")

In [ ]:
_ = xlab.tests.square.task4(eta)

### Task 5
Now code the $L_2$ distribution using the above helper functions. This will use the eta function to create a square composed of two rectangles, one positive and one negative. This square is then randomly placed on the pixels of the image, and a scaled using a random scalar, to create a perturbation delta.

$$
\begin{array}{l}
\text{\bf Algorithm: Sampling distribution } P \text{ for } l_2\text{-norm} \\
\hline
\text{\bf Input: } \text{maximal norm } \epsilon, \text{ window size } h, \text{ image size } w, \text{ number of color channels } c, \\
\quad \text{current image } \hat{x}, \text{ original image } x \\
\text{\bf Output: } \text{New update } \delta \\[0.5em]
\nu \leftarrow \hat{x} - x \\
\text{sample uniformly } r_1, s_1, r_2, s_2 \in \{0, \dots, w - h\} \\
W_1 := r_1 + 1 : r_1 + h, s_1 + 1 : s_1 + h, W_2 := r_2 + 1 : r_2 + h, s_2 + 1 : s_2 + h \\
k \gets \left\lfloor \frac{h}{2} \right\rfloor \\
\eta \gets \text{randomly choose } \left( \eta^{(h, \ k)}, -\eta^{(h, \ h-k)} \right) \text{ or } \left( \eta^{(h, \ k)}, -\eta^{(h, \ h-k)} \right)^\mathsf{T} \\
\eta^* \leftarrow \frac{\eta}{\lVert \eta \lVert_2} \\
\epsilon^2_{\text{unused}} \leftarrow \epsilon^2 - \lVert \nu \lVert_2^2 \\ 
\text{\bf for } i = 1, \dots, c \text{ \bf do } \\
\quad \rho \leftarrow \text{Uniform}(\{-1, 1\}) \\
\quad \nu_{\text{temp}} \leftarrow \rho\eta^* + \frac{\nu_{W_1, i}}{\lVert \nu_{W_1, i} \lVert_2} \\
\quad \epsilon^i_{\text{avail}} \leftarrow \sqrt{\lVert \nu_{W_1 \cup W_2, i} \lVert_2^2 + \frac{\epsilon^2_{\text{unused}}}{c}} \\
\quad \nu_{W_2, i} \leftarrow 0, \quad \nu_{W_1, i} \leftarrow \left( \frac{\nu_{\text{temp}}}{\lVert \nu_{\text{temp}} \lVert_2} \right) \epsilon^i_{\text{avail}} \\
\text{\bf end } \\
\delta \leftarrow x + \nu - \hat{x} \\
\hline
\end{array}
$$


<details>
<summary>💡 <b>Hint for Task #5</b></summary>

Cloning v at the beginning can avoid a logic error (delta updates are 0 every time)
</details>

<details>
<summary>💡 <b>Hint for Task #5</b></summary>

Adding a very small number to calculation denominators can avoid nan or infinity errors.
</details>


<details>
<summary>🔐 <b>Solution for Task #5</b></summary>

```python
def l_2_dist(x_hat, x, epsilon = 1/1000, h = 10, w = 32, c = 3):
    """Generates L2-constrained perturbation update by redistributing energy between random square patches
    
    Args:
        x_hat [1, c, w, w]: Current perturbed image tensor.
        x [1, c, w, w]: Original clean image tensor.
        epsilon (float): L2 perturbation budget constraint.
        h (int): Height and width of square patches.
        w (int): Image width and height.
        c (int): Number of color channels.
    
    Returns:
        [c, w, w]: Perturbation update tensor that redistributes energy while respecting L2 constraints.
    """
    v = x_hat - x
    v_new = v.clone()
    
    delta = torch.zeros(c,w,w)
    r1, s1, r2, s2 = np.random.randint(w-h, size = (4))
    v_sq = v ** 2
    eps_unused = max(0, epsilon ** 2 - torch.sum(v_sq))
    k = h // 2
    
    e1 = torch.hstack((eta(h, k), -1 * eta(h, h-k)))
    e2 = torch.hstack((eta(h, k), -1 * eta(h, h-k))).T
    e = random.choice([e1,e2])
    eta_star = e / (torch.norm(e, p=2) + 1e-9)
    
    for i in range(c):
        p = np.random.uniform(-1, 1)
        W1 = v[0][i, r1:r1+h, s1:s1+h]
        W2 = v[0][i, r2:r2+h, s2:s2+h]
        
        v_t = p * eta_star + W1 / (torch.norm(W1) + 1e-9)
        e_avail = torch.sqrt(torch.sum(W1 ** 2) + torch.sum(W2 ** 2) + eps_unused / c)   
        
        v_new[0, i, r2:r2+h, s2:s2+h] = 0
        v_new[0, i, r1:r1+h, s1:s1+h] = (v_t / (torch.norm(v_t) + 1e-9)) * e_avail

    delta =  v_new - v
    return delta
```

</details>

In [ ]:
def l_2_dist(x_hat, x, epsilon = 1/1000, h = 10, w = 32, c = 3):
    """Generates L2-constrained perturbation update by redistributing energy between random square patches
    
    Args:
        x_hat [1, c, w, w]: Current perturbed image tensor.
        x [1, c, w, w]: Original clean image tensor.
        epsilon (float): L2 perturbation budget constraint.
        h (int): Height and width of square patches.
        w (int): Image width and height.
        c (int): Number of color channels.
    
    Returns:
        [c, w, w]: Perturbation update tensor that redistributes energy while respecting L2 constraints.
    """
    
    raise NotImplementedError("l_2_dist hasn't been implemented.")

In [ ]:
_ = xlab.tests.square.task5(l_2_dist)

Now we implement the main loop for $L_2$ square attack. This is the same as the above $L_\infty$, but uses the other distribution.
The $L_2$ Attack is mathematically much slower than the $L_\infty$ attack. Epsilon has been increased to ensure that the algorithm can run reasonably quickly. 

In [ ]:
def l_2_square_attack(model, loss_fn, x, y, N, w=32, c=3, epsilon=100/1000, max_h = 10):
    x_hat = x.clone()
    loss = loss_fn(model(x), y)
    i = 1
    h = max_h
    while i < N and prediction(model, x_hat)[0] == y:
        if i % (N // max_h) == 0:
            if h > 2:
                h -= 1
        delta = l_2_dist(x_hat, x, h=h, epsilon=epsilon, w=w, c=c)
        x_new = x_hat + delta
        x_new = torch.clamp(x_new, 0, 1)
        loss_new = loss_fn(model(x_new), y)
        if loss_new > loss:
            loss = loss_new
            x_hat = x_new 
        i += 1
    return x_hat

img = process_image(IMG_PATH)
label = prediction(model, process_image(IMG_PATH))[0]
loss_fn = torch.nn.CrossEntropyLoss()
x_adv = l_2_square_attack(model, loss_fn, img, label, 5000)
pred = prediction(model, x_adv)
print(f"{classes[pred[0]]} with probability {pred[1][0]:.4f}")
show_image(x_adv)